# **Youtube Informative-video Summerizer**

This python app let's you summerize youtube videos that contains information-sharing-through-talking, like someone talking about a subject, someone sharing a life advice, a podcast etc.

We extract the transcipt analyize it with an LLM to summerize and create summerization and analysis.


> We use youtube_transcript_api which allows you to get the transcript text of any youtube video.

> Results however are not ideal for our use case since it does not provide who says what in case of more than one speaker. it only provide one giant string of all the words said in the video respectivly with some noise.










In [5]:
uv add youtube-transcript-api

/Users/bogdanteslarasu/workspace/repos/llm/llm_engineering/.venv/bin/python: No module named uv
Note: you may need to restart the kernel to use updated packages.


In [9]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter, SRTFormatter
import re
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

In [10]:
ytt = YouTubeTranscriptApi()
formatter = TextFormatter() # --> Plain text
# formatter = SRTFormatter() # --> With timestamps

openai_api_key = os.getenv('OPENAI_API_KEY')
openai_client = OpenAI(api_key=openai_api_key)
MODEL = "gpt-4o-mini"

In [11]:
def get_video_id(url):
    """Extracts video ID from a YouTube URL."""
    regex = r"(?:v=|\/)([0-9A-Za-z_-]{11}).*"
    match = re.search(regex, url)
    if match:
        return match.group(1)
    raise ValueError("Invalid YouTube URL")


def get_transcript(url):
    video_id = get_video_id(url)
    fetched_transcript = ytt.fetch(video_id)
    # ^ defaults to English transcript, for other language use:
    # fetched = ytt.fetch(video_id, languages=['de', 'en'])
    transcript_text = formatter.format_transcript(fetched_transcript)
    return transcript_text

In [12]:
system_prompt = """You are an expert assistant specialized in analyzing podcast transcripts. You will be given the full transcript of a YouTube podcast episode.

Your task is to extract and summarize the main views or arguments presented in the podcast. For each view or argument, also identify and list any supporting evidence such as:

- Facts or statistics
- Academic studies or research
- Theories or philosophical frameworks
- Anecdotes or personal experiences
- Expert opinions or quotes

Recognize off topic segments and adds and igrone them.

Structure your output in a clear and concise format.

Output Format:

Podcast Summary:

1. View/Argument:
   - Description: [Summarize the view or claim in 1-2 sentences.]
   - Supporting Evidence:
     • [Fact, study, or reasoning #1]
     • [Fact, study, or reasoning #2]
     • [Optional counterarguments or nuances, if any]

2. View/Argument:
   - Description: [...]
   - Supporting Evidence:
     • [...]

Guidelines:
- Only include major views or arguments that are discussed in depth.
- Paraphrase in clear, neutral, and objective language.
- Do not include filler, small talk, or off-topic segments.
- If a claim lacks explicit evidence, note it as “No clear supporting evidence provided.”

Always respond and orginize your response using Markdow.
"""


In [13]:
def get_user_prompt(title,url):
    prompt = f"Following is a transcript for a podcast titled '{title}' \n"
    prompt += "Carefully read through this content, analyse and summerize it as told, respond in Markdown."
    prompt += "\nTranscript: \n\n"
    prompt += get_transcript(url)
    return prompt

In [14]:
# user_prompt = get_user_prompt()
def summerize_video(title,url):
    user_prompt = get_user_prompt(title,url)
    stream = openai_client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        stream = True,
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [15]:
summerize_video("You Need to Be Bored. Here's Why.","https://www.youtube.com/watch?v=orQKfIXMiA8")

# Podcast Summary: You Need to Be Bored. Here's Why.

1. **View/Argument: Boredom is essential for mental well-being.**
   - **Description:** The speaker argues that boredom encourages the brain to engage in introspection, leading to deeper understanding of one's life and reducing feelings of emptiness and depression.
   - **Supporting Evidence:**
     • The "default mode network" in the brain activates during boredom, prompting reflection on significant existential questions.
     • Many participants in a Harvard study chose to administer painful shocks to avoid boredom, indicating a strong aversion to uncomfortable thoughts.
     • Increased rates of anxiety and depression in society correlate with a reduced capacity for individuals to find meaning in their lives.
     
2. **View/Argument: Modern technology exacerbates the dislike of boredom.**
   - **Description:** The prevalence of smartphones and screens allows individuals to escape boredom, hindering their ability to grapple with meaningful questions and fostering a cycle of dissatisfaction.
   - **Supporting Evidence:**
     • The speaker attributes the increase of depression and anxiety to the avoidance of boredom through constant engagement with devices.
     • Engaging more fully in moments of boredom can lead to greater appreciation of daily life, relationships, and work.
   
3. **View/Argument: Implementing personal protocols can combat device dependency and promote boredom.**
   - **Description:** The speaker advocates for self-imposed strategies to limit device use, enabling individuals to embrace boredom and enhance introspection.
   - **Supporting Evidence:**
     • Personal anecdotes describing a "no device policy" after 7 PM, family meals without devices, and regular social media fasts illustrate effective measures to combat technology addiction.
     • The speaker experiences initial discomfort during device abstinence, which eventually leads to feelings of calmness and well-being.
     • Practical suggestions for handling emergencies with device use highlight the non-essential nature of constant connectivity.
   
4. **View/Argument: Longer periods of intentional boredom can facilitate personal growth.**
   - **Description:** The speaker encourages listeners to practice extending boredom to gain insights into life's bigger questions, potentially leading to increased happiness.
   - **Supporting Evidence:**
     • By deliberately experiencing boredom, individuals can develop resilience against mundane aspects of life and foster personal reflection.
     • Suggestions for practice include gym workouts and commutes without distractions, enhancing the capacity for self-exploration and creativity. 

This structured analysis captures the key arguments presented in the podcast while highlighting supporting evidence and personal experiences.